# Shipping Lanes Case Study

## Background

When working with radar data, water commonly appears dark; its relatively smooth surface results in very little backscatter, and consequently, low intensities recorded by the satellite in both polarisation bands. However, if a large ship is on the water, the backscatter at the ship's location will be much higher than the water.

In this notebook, we attempt to identify shipping lanes around Australia by taking advantage of the fact that ships on the water result in a significant backscatter signal. This can be done by finding the maximum of each pixel in Sentinel-1 images. Computing the maximum for a large number of pixel is computationally expensive, so we make use of the parallel-computing library [Dask](https://docs.dask.org/en/latest/).

This notebook was inspired by [this thread](https://twitter.com/sgascoin/status/1113184052587171842) on Twitter.

## Import key packages for analysis

In [ ]:
%matplotlib inline

import datacube
import datetime
import numpy as np
import json
import matplotlib.pyplot as plt
from utils.display import display_map

dc = datacube.Datacube(app='dc-visualize')

## Set up distributed computing
Access to Dask is provided as part of the DEA Sandbox environment. The cluster for parallel computing is set up by calling the `Client()` function.

In [ ]:
import dask
from dask.distributed import Client

client = Client('dask-datacube-dask.labs:8786')

## Select and view a study area

There are a number of key ports covered by DEA Sentinel-1 data. The available data can be viewed on the [Dashboard](https://dashboard.dea-sandbox.test.frontiersi.io/s1_gamma0_geotif_scene), but we also list the latitude and longitude coordinates for a few key ports below.

### New South Wales
**Port of Newcastle**<br>
This is one of Australia's largest ports, with a diverse range of cargo types. <br>
`latitude, longitude = (-32.9267, 151.7821)`

**Port Kembla**<br>
This is the home to New South Wales' largest motor vehicle import hub and grain export terminal. <br>
`latitude, longitude = (-34.4854, 150.8996)`

**Port Jackson**<br>
This includes Sydney Harbour, which houses numerous cruise ships and ferries. <br>
`latitude, longitude = (-33.8462, 151.2489)`

### Northern Territory

**Port Darwin**<br>
This is a key port in Darwin, covering livestock, containers, and cruise ships. <br>
`latitude, longitude = (-12.4167, 130.8000)`

### Tasmania

**Devonport**<br>
This is where the Spirit of Tasmania (the ferry between Melbourne and Tasmania) transits from. <br>
`latitude, longitude = (-41.1771, 146.3452)`

The cell below can be used to specify the study area and time frame, then load a map of the study area. The area is selected by adding and subtracting `buffer` in degrees around the central latitude and longitude. Experiment with the value of this parameter to best capture the ship activity in the selected port.

In [ ]:
# Set the centre latitude and longitude coordinates
latitude, longitude = (-41.1771, 146.3452)

# Set the study area around the centre coordinates, where buffer is in degrees
buffer = 0.2
latitude_range = (latitude - buffer, latitude + buffer)
longitude_range = (longitude - buffer, longitude + buffer)

# Set the time frame
date_range = ('2017-01-01', '2018-01-01')

display_map(latitude=latitude_range, longitude=longitude_range)

## Load and view the data

Data is loaded with the `dc.load()` function. The Sentinel-1 product `s1_gamma0_geotif_scene` has two measurement bands associated with it, `vh` and `vv`, corresponding to the cross-polarisation and co-polarisation measurements from the satellite. The `dask_chunks` argument is used to tell Dask how to segement the data for parallelised computations (see the [xarray documentation](http://xarray.pydata.org/en/stable/dask.html)). 

In [ ]:
%%time

ds = dc.load(
    product='s1_gamma0_geotif_scene',
    x=longitude_range,
    y=latitude_range,
    output_crs='epsg:3577',
    resolution=(-25, 25),
    time = date_range,
    measurements=['vh', 'vv'],
    dask_chunks = {'x': 4000, 'y': 4000, 'time': 1}
)

print(ds)

## Calculating and plotting the maximum pixel value

After loading the data, we can calculate the maximum value for each pixel, additionally converting from linear scale to decibel scale by multiplying the base-10 logarithm by 10. Note that the Dask operation will occur when attempting to plot the `vh` band in the cell below. To see the resources being used when you run the operation, visit https://dask.sandbox.dea.ga.gov.au/status.

In [ ]:
%%time

maximum = 10*np.log10(ds.max(dim='time'))

maximum.vh.plot.imshow(size=10, cmap="Greys_r", robust=True)
plt.show()

## Conclusion

The high contrast between the ships and the water can make for striking images. Fortunately, processing with Dask makes it possible to make these computationally-intensive images in under a minute, even for very large areas.

Only a year of data is available at the moment, but more observations would likely lead to clearer identification of the shipping lanes.